# Post Selection

We can do some error mitigation by throwing out the states with the wrong number of particles.  To achieve this we need a rotation in which both the total number operator $$n = \sum_i n_i \rightarrow \sum_i (I - Z_i)$$ and the Kinetic terms $$K_i = c^{\dagger}_ic_{i+1} + c^{\dagger}_{i+1}c_{i} \rightarrow X_iX_{i+1} + Y_iY_{i+1}$$ are diagonal.

In [340]:
import qiskit.quantum_info as qi
import scipy as sp
import scipy.linalg as ln
import numpy as np
import pandas as pd
import math


def Mdot(Ol):
    out = Ol[0]
    for i in range(1,len(Ol)):
        out = np.dot(Ol[i],out)
    return out

def bkt(y1,O,y2):
    return Mdot([np.conjugate(y1),O,y2])


def X(i,N):
    label = ['I' for i in range(N)]
    label[N-1-i] = 'X'
    label = ''.join(label)
    return qi.Operator.from_label(label).data

def Y(i,N):
    label = ['I' for i in range(N)]
    label[N-1-i] = 'Y'
    label = ''.join(label)
    return qi.Operator.from_label(label).data

def Z(i,N):
    label = ['I' for i in range(N)]
    label[N-1-i] = 'Z'
    label = ''.join(label)
    return qi.Operator.from_label(label).data

def c(i,N):
    label_1 = ['Z' for j in range(N-i-1)]
    label_2 = ['I' for j in range(N-i,N)]
    label_x = label_1 + ['X'] + label_2
    label_y = label_1 + ['Y'] + label_2
    label_x = ''.join(label_x)
    label_y = ''.join(label_y)
    x = qi.Operator.from_label(label_x).data
    y = qi.Operator.from_label(label_y).data
    return 1/2*(x+1j*y)

def cd(i,N):
    label_1 = ['Z' for j in range(N-i-1)]
    label_2 = ['I' for j in range(N-i,N)]
    label_x = label_1 + ['X'] + label_2
    label_y = label_1 + ['Y'] + label_2
    label_x = ''.join(label_x)
    label_y = ''.join(label_y)
    x = qi.Operator.from_label(label_x).data
    y = qi.Operator.from_label(label_y).data
    return 1/2*(x-1j*y)

import numpy as np

def Mdot(Ol):
    L = len(Ol)
    out = Ol[L-1]
    for i in range(1,len(Ol)):
        out = np.dot(Ol[L-1-i],out)
    return out

def bkt(y1,O,y2):
    return Mdot([np.conjugate(y1),O,y2])

# Using operators

This rotation will be composed of $$\sqrt{i\text{SWAP}}_{ij} = e^{i \frac{\pi}{2}(X_iX_j + Y_iY_j)} $$ and $$T_i = e^{i\frac{\pi}{8}Z_i}$$

The complete rotation is $$ U_{ij} = \sqrt{i\text{SWAP}}_{ij}T_iT^{\dagger}_j $$

In [341]:
riswap = 1/2*(I(2) + Mdot([Z(0,2),Z(1,2)]) + np.cos(np.pi/4)*(I(2) - Mdot([Z(0,2),Z(1,2)])) + 1j*np.sin(np.pi/4)*(Mdot([X(0,2),X(1,2)]) + Mdot([Y(0,2),Y(1,2)])))

riswap_tst = ln.expm(1j*np.pi/8*( Mdot([X(0,2),X(1,2)]) + Mdot([Y(0,2),Y(1,2)]) ))


np.amax(np.abs( riswap - riswap_tst ))

1.1102230246251565e-16

In [342]:
riswap_tst = ln.expm(1j*np.pi/8*( Mdot([X(0,2),X(1,2)]) + Mdot([Y(0,2),Y(1,2)]) ))
T0 = np.cos(np.pi/8)*I(2) + 1j*np.sin(np.pi/8)*Z(0,2)
T1 = np.cos(np.pi/8)*I(2) - 1j*np.sin(np.pi/8)*Z(1,2)

Um = Mdot([riswap,T0,T1])
Umd = np.conjugate(np.transpose(Um))

pd.DataFrame( Mdot([Um, 1/2*(Mdot([X(0,2),X(1,2)]) + Mdot([Y(0,2),Y(1,2)]) ) ,Umd]) )

,0,1,2,3
0,0.0+0.0j,0.000000e+00+0.000000e+00j,0.000000e+00+0.000000e+00j,0.0+0.0j
1,0.0+0.0j,-1.000000e+00+0.000000e+00j,-2.220446e-16-2.220446e-16j,0.0+0.0j
2,0.0+0.0j,-2.220446e-16+2.220446e-16j,1.000000e+00+0.000000e+00j,0.0+0.0j
3,0.0+0.0j,0.000000e+00+0.000000e+00j,0.000000e+00+0.000000e+00j,0.0+0.0j


In [343]:
(Z(0,2) - Z(1,2))/2

array([[ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j]])

In [344]:
pd.DataFrame( Mdot([Um, Z(0,2) + Z(1,2) ,Umd]) )

,0,1,2,3
0,2.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j
1,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j
2,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j
3,0.0+0.0j,0.0+0.0j,0.0+0.0j,-2.0+0.0j


In order to perform the $\sqrt{i\text{SWAP}}_{ij}$ gate using only two qubit gates we either have to expand $\sqrt{i\text{SWAP}}_{ij}$ in terms of many gates or we can re-run the slatter determinate circuit with new qubit indcices so that $i$ and $j$ are neighbors in the quantum hardware.  

## $\sqrt{\text{iSWAP}}$ in terms of CNOT gates

We need to be able to contruct $\sqrt{\text{iSWAP}}$ on the quantum computer using basic gates.  I found the following on a message board.

$$ \sqrt{\text{iSWAP}} = C^X_{10}R^{Y}_1(\frac{\pi}{2}) R^{Z}_1(\frac{\pi}{8}) C^X_{01} R^{Z}_1(-\frac{\pi}{4}) C^X_{01} R^{Z}_1(\frac{\pi}{8}) R^{Y}_1(-\frac{\pi}{2})$$

Let's test it.

In [352]:
def Rz(phi,i,N):
    return np.cos(phi/2)*I(N) - 1j*np.sin(phi/2)*Z(i,N) 

def Ry(phi,i,N):
    return np.cos(phi/2)*I(N) - 1j*np.sin(phi/2)*Y(i,N) 

def Rx(phi,i,N):
    return np.cos(phi/2)*I(N) - 1j*np.sin(phi/2)*X(i,N) 

def Cx(i,j,N):
    return 1/2*( I(N) + Z(i,N) + X(j,N) - Mdot([Z(i,N),X(j,N)]) )

def Cxm(i,j,N):
    return 1/2*( I(N) + Z(i,N) - X(j,N) + Mdot([Z(i,N),X(j,N)]) )



riswap_tst2 = Mdot([Cx(1,0,2),Ry(np.pi/2,1,2),Rz(-np.pi/8,1,2),Cx(0,1,2),Rz(np.pi/4,1,2),Cx(0,1,2),Rz(-np.pi/8,1,2),Ry(-np.pi/2,1,2),Cx(1,0,2)])

np.amax(np.abs( riswap_tst2-riswap ))


2.2291027918243403e-16

In [353]:
np.amax(np.abs( Mdot([riswap_tst2,Rz(-np.pi/4,0,2),Rz(np.pi/4,1,2)]) - Um ))

2.2291027918243403e-16

# Using gates

In [388]:
from qiskit import QuantumCircuit, transpile, QuantumRegister,ClassicalRegister, execute
import copy

def apply_riswap(qc_in,j,i):
    qc = copy.deepcopy(qc_in)
    qc.cx(j,i)
    qc.ry(np.pi/2,j)
    qc.rz(np.pi/8,j)
    qc.cx(i,j)
    qc.rz(-np.pi/4,j)
    qc.cx(i,j)
    qc.rz(np.pi/8,j)
    qc.ry(-np.pi/2,j)
    qc.cx(j,i)
    return qc

def apply_Um(qc_in,j,i):
    qc = copy.deepcopy(qc_in)
    qc.rz(-np.pi/4,j)
    qc.rz(np.pi/4,i)
    qc = apply_riswap(qc,i,j)
    return qc

qr = QuantumRegister(2)
cr = ClassicalRegister(2)
qc = QuantumCircuit(qr , cr)
qc = apply_Um(qc,0,1)
qc.draw()

┌──────────┐┌───┐                                            »
q114_0: ┤ Rz(-π/4) ├┤ X ├────────────────────────■────────────────■──»
        ├─────────┬┘└─┬─┘┌─────────┐┌─────────┐┌─┴─┐┌──────────┐┌─┴─┐»
q114_1: ┤ Rz(π/4) ├───■──┤ Ry(π/2) ├┤ Rz(π/8) ├┤ X ├┤ Rz(-π/4) ├┤ X ├»
        └─────────┘      └─────────┘└─────────┘└───┘└──────────┘└───┘»
c114: 2/═════════════════════════════════════════════════════════════»
                                                                     »
«                               ┌───┐
«q114_0: ───────────────────────┤ X ├
«        ┌─────────┐┌──────────┐└─┬─┘
«q114_1: ┤ Rz(π/8) ├┤ Ry(-π/2) ├──■──
«        └─────────┘└──────────┘     
«c114: 2/════════════════════════════
«

In [390]:
import qiskit.quantum_info as qi

#define a random state
qr = QuantumRegister(2)
cr = ClassicalRegister(2)
circ = QuantumCircuit(qr , cr)

circ.ry(0.33,0)
circ.x(0)
circ.cx(0,1)
circ.y(0)

psi = qi.Statevector.from_instruction(circ)

pnum = bkt(psi,1/2*(I(2)-Z(0,2)) + 1/2*(I(2)-Z(1,2)),psi)

kexp = bkt(psi, 1/2*(Mdot([X(0,2),X(1,2)])+Mdot([Y(0,2),Y(1,2)])) ,psi)

circ2 = apply_Um(circ,0,1)

psi2 = qi.Statevector.from_instruction(circ2)

results = psi2.probabilities_dict()

print(kexp)
results

(-0.32404302839486837+0j)


{'01': 0.6620215141974342, '10': 0.3379784858025657}

In [394]:
np.amax(np.abs( Mdot([Um,psi]) - psi2.data ))

2.0014830212433605e-16

Notice that the particle number is conserved so any weights for states outside of the correct particle number can be dropped.  

Under this rotation, $$ \frac{1}{2}(XX + YY) \rightarrow \frac{1}{2}(Z_0 - Z_1) $$

In [395]:
Z0exp = -results['01'] + results['10']
Z1exp = results['01'] - results['10']

In [396]:
1/2*(Z0exp - Z1exp)

-0.3240430283948685

# Dealing with non-local XX+YY

Let us use the slatter determinant as the trial wavefunction 

## Using Slatter circuit

In [397]:


def F(K):
    e,y = ln.eigh(K)
    return np.transpose(y)
    
def Fd(K):
    e,y = ln.eigh(K)
    return np.conjugate(y)

In [488]:
from qiskit import QuantumCircuit, transpile, QuantumRegister,ClassicalRegister, execute

def ry(i,j,phi,N):
    M = (1+0*1j)*np.identity(N)
    M[i,i] = np.cos(phi)
    M[j,j] = np.cos(phi)
    M[i,j] = np.sin(phi)
    M[j,i] = -np.sin(phi)
    return M


def rz(j,phi,N):
    M = (1+0*1j)*np.identity(N)
    M[j,j] = np.exp(1j*phi)
    return M


from qiskit import QuantumCircuit

def fswap(i,j,qc):
    qc.swap(i,j)
    qc.ry(np.pi/2,j)
    qc.cx(i,j)
    qc.ry(-np.pi/2,j)
    return qc
    

def R_cc(i,j,phi,qc):
    for l in range(i+1,j):
        qc = fswap(l-1,l,qc)
    qc.ry(-np.pi/2,j-1)
    qc.cx(j-1,j)
    qc.ry(-phi,j)
    qc.cx(j-1,j)
    qc.ry(np.pi/2,j-1)
    qc.ry(-np.pi/2,j)
    qc.cx(j,j-1)
    qc.ry(phi,j-1)
    qc.cx(j,j-1)
    qc.ry(np.pi/2,j)
    for l in range(j-1,i+1-1,-1):
        qc = fswap(l-1,l,qc)
    return qc

def G_cc(i,j,phi,phiz,qc):
    qc = R_cc(i,j,-phi,qc)
    qc.rz(-phiz,j)
    return qc

#Had to modify phiz so that it took imaginary values
def givens(ri,rj,c,F):
    N = len(F)
    if F[rj,c] == 0:
        F_new = F
        phiz = 0
        phi = 0
    elif F[ri,c] == 0:
        phiz = 1j*np.log( F[rj,c]/np.abs(F[rj,c]) )
        Fz =  Mdot([rz(rj,phiz,N) , F])
        phi = np.pi/2
        F_new = Mdot([ry(ri,rj,phi,N) , Fz])
    else:
        phiz = 1j*np.log( F[rj,c]/F[ri,c] * np.abs(F[ri,c])/np.abs(F[rj,c]) +0*1j)
        Fz =  Mdot([rz(rj,phiz,N) , F])
        phi = np.arctan(Fz[rj,c]/Fz[ri,c])
        F_new = Mdot([ry(ri,rj,phi,N) , Fz])
    return F_new,phiz,phi

def slatter_circ(F0):
    F1,pz1,p1 = givens(2,3,0,F0)
    F2,pz2,p2 = givens(1,2,0,F1)
    F3,pz3,p3 = givens(0,1,0,F2)
    F4,pz4,p4 = givens(2,3,1,F3)
    F5,pz5,p5 = givens(1,2,1,F4)
    F6,pz6,p6 = givens(2,3,2,F5)
    ph0 = -1j*np.log(F6[0,0])
    ph1 = -1j*np.log(F6[1,1])
    ph2 = -1j*np.log(F6[2,2])
    ph3 = -1j*np.log(F6[3,3])
    qr = QuantumRegister(4)
    cr = ClassicalRegister(4)
    qc = QuantumCircuit(qr , cr)
    qc.x(2)
    qc.x(3)
    qc.rz(np.real(ph0),0)
    qc.rz(np.real(ph1),1)
    qc.rz(np.real(ph2),2)
    qc.rz(np.real(ph3),3)
    qc = G_cc(2,3,np.real(p6),np.real(pz6),qc)
    qc = G_cc(1,2,np.real(p5),np.real(pz5),qc)
    qc = G_cc(2,3,np.real(p4),np.real(pz4),qc)
    qc = G_cc(0,1,np.real(p3),np.real(pz3),qc)
    qc = G_cc(1,2,np.real(p2),np.real(pz2),qc)
    qc = G_cc(2,3,np.real(p1),np.real(pz1),qc)
    return qc

In [489]:
def K_single_triag(k):
    h = [[0 for i in range(4)] for ii in range(4)]
    h[0][1] = -k; h[0][2] = -k; h[0][3] = -k;
    h[1][0] = -k; h[1][2] = -k; h[1][3] = -k;
    h[2][0] = -k; h[2][1] = -k; h[2][3] = -k;
    h[3][0] = -k; h[3][1] = -k; h[3][2] = -k;
    return h

K = K_single_triag(1.2)
circ = slatter_circ(Fd(K))
psi = qi.Statevector.from_instruction(circ)

pnum = bkt(psi,1/2*(I(4)-Z(0,4)) + 1/2*(I(4)-Z(1,4)) + 1/2*(I(4)-Z(2,4)) + 1/2*(I(4)-Z(3,4)),psi)

kexp = bkt(psi, 1/2*(Mdot([X(0,4),X(1,4)])+Mdot([Y(0,4),Y(1,4)])) ,psi)

circ2 = apply_Um(circ,0,1)
psi2 = qi.Statevector.from_instruction(circ2)
results = psi2.probabilities_dict()

pnum2 = bkt(psi2,1/2*(I(4)-Z(0,4)) + 1/2*(I(4)-Z(1,4)) + 1/2*(I(4)-Z(2,4)) + 1/2*(I(4)-Z(3,4)),psi2)

print(pnum,pnum2)
print(kexp)
print()
#print(results)

(1.9999999999999991+0j) (1.9999999999999987+0j)
(-4.8316687112850974e-17+0j)



In [434]:
Z0exp = -results['0011'] - results['0101'] - results['1001'] + results['0110'] + results['1010'] + results['1100']
Z1exp = -results['0011'] + results['0101'] + results['1001'] - results['0110'] - results['1010'] + results['1100']

1/2*(Z0exp - Z1exp)

-5.551115123125783e-17

In [435]:
N=4 

riswap_tst = ln.expm(1j*np.pi/8*( Mdot([X(0,N),X(1,N)]) + Mdot([Y(0,N),Y(1,N)]) ))
T0 = np.cos(np.pi/8)*I(N) + 1j*np.sin(np.pi/8)*Z(0,N)
T1 = np.cos(np.pi/8)*I(N) - 1j*np.sin(np.pi/8)*Z(1,N)
Um = Mdot([riswap_tst,T0,T1])
Umd = np.conjugate(np.transpose(Um))

np.amax(np.abs( (Z(0,N) - Z(1,N)) - Mdot([Um, Mdot([X(0,N),X(1,N)])+Mdot([Y(0,N),Y(1,N)]), Umd])  ))

7.711855592701269e-16

In [436]:
np.amax(np.abs( Mdot([Um,psi]) - psi2.data ))

7.850462293418876e-17

## Non-local XX+YY

### Without reorientation

In [529]:
def K_single_triag(k):
    h = [[0 for i in range(4)] for ii in range(4)]
    h[0][1] = -k; h[0][2] = -k; h[0][3] = -k;
    h[1][0] = -k; h[1][2] = -k; h[1][3] = -k;
    h[2][0] = -k; h[2][1] = -k; h[2][3] = -k;
    h[3][0] = -k; h[3][1] = -k; h[3][2] = -k;
    return h

i = 1
j = 3

K = K_single_triag(1.2)
circ = slatter_circ(Fd(K))
psi = qi.Statevector.from_instruction(circ)

pnum = bkt(psi,1/2*(I(4)-Z(0,4)) + 1/2*(I(4)-Z(1,4)) + 1/2*(I(4)-Z(2,4)) + 1/2*(I(4)-Z(3,4)),psi)

kexp = bkt(psi, 1/2*(Mdot([X(i,4),X(j,4)])+Mdot([Y(i,4),Y(j,4)])) ,psi)

circ2 = apply_Um(circ,i,j)
psi2 = qi.Statevector.from_instruction(circ2)
results = psi2.probabilities_dict()

pnum2 = bkt(psi2,1/2*(I(4)-Z(0,4)) + 1/2*(I(4)-Z(1,4)) + 1/2*(I(4)-Z(2,4)) + 1/2*(I(4)-Z(3,4)),psi2)

print(pnum,pnum2)
print(kexp)

(1.9999999999999991+0j) (1.9999999999999987+0j)
(0.6666666666666664+0j)


In [531]:
Z0exp = -results['0011'] - results['0101'] - results['1001'] + results['0110'] + results['1010'] + results['1100']
Z1exp = -results['0011'] + results['0101'] + results['1001'] - results['0110'] - results['1010'] + results['1100']
Z2exp = results['0011'] - results['0101'] + results['1001'] - results['0110'] + results['1010'] - results['1100']
Z3exp = results['0011'] + results['0101'] - results['1001'] + results['0110'] - results['1010'] - results['1100']


1/2*(Z1exp - Z3exp)

0.6666666666666663

### With reorientation

In [560]:
def swap_idx(i,j,Op):
    S = np.identity(len(Op))
    S[i,i] = 0
    S[j,j] = 0
    S[i,j] = 1
    S[j,i] = 1
    return Mdot([S,Op])

In [561]:
pd.DataFrame( swap_idx(1,3,Fd(K)) )

,0,1,2,3
0,0.5,0.866025,0.000000,0.000000e+00
1,0.5,-0.288675,-0.408248,7.071068e-01
2,0.5,-0.288675,-0.408248,-7.071068e-01
3,0.5,-0.288675,0.816497,8.756053e-17


In [562]:
pd.DataFrame( Fd(K) )

,0,1,2,3
0,0.5,0.866025,0.000000,0.000000e+00
1,0.5,-0.288675,0.816497,8.756053e-17
2,0.5,-0.288675,-0.408248,-7.071068e-01
3,0.5,-0.288675,-0.408248,7.071068e-01


In [571]:
K = K_single_triag(1.2)
Fswap = swap_idx(1,3,Fd(K))
circ = slatter_circ(Fswap)
psi = qi.Statevector.from_instruction(circ)
probs = psi.probabilities_dict()
probs['1010']

0.33333333333333315

In [567]:
K = K_single_triag(1.2)
circ = slatter_circ(Fd(K))
psi = qi.Statevector.from_instruction(circ)
probs = psi.probabilities_dict()
probs['0110']

0.33333333333333315

In [572]:
def K_single_triag(k):
    h = [[0 for i in range(4)] for ii in range(4)]
    h[0][1] = -k; h[0][2] = -k; h[0][3] = -k;
    h[1][0] = -k; h[1][2] = -k; h[1][3] = -k;
    h[2][0] = -k; h[2][1] = -k; h[2][3] = -k;
    h[3][0] = -k; h[3][1] = -k; h[3][2] = -k;
    return h

i = 1
j = 3

K = K_single_triag(1.2)
Fswap = swap_idx(1,j,swap_idx(0,i,Fd(K)))
circ = slatter_circ(Fswap)
psi = qi.Statevector.from_instruction(circ)

pnum = bkt(psi,1/2*(I(4)-Z(0,4)) + 1/2*(I(4)-Z(1,4)) + 1/2*(I(4)-Z(2,4)) + 1/2*(I(4)-Z(3,4)),psi)

kexp = bkt(psi, 1/2*(Mdot([X(0,4),X(1,4)])+Mdot([Y(0,4),Y(1,4)])) ,psi)

circ2 = apply_Um(circ,0,1)
psi2 = qi.Statevector.from_instruction(circ2)
results = psi2.probabilities_dict()

pnum2 = bkt(psi2,1/2*(I(4)-Z(0,4)) + 1/2*(I(4)-Z(1,4)) + 1/2*(I(4)-Z(2,4)) + 1/2*(I(4)-Z(3,4)),psi2)

print(pnum,pnum2)
print(kexp)


(1.9999999999999987+0j) (1.9999999999999982+0j)
(-0.6666666666666662+0j)


In [573]:
Z0exp = -results['0011'] - results['0101'] - results['1001'] + results['0110'] + results['1010'] + results['1100']
Z1exp = -results['0011'] + results['0101'] + results['1001'] - results['0110'] - results['1010'] + results['1100']
Z2exp = results['0011'] - results['0101'] + results['1001'] - results['0110'] + results['1010'] - results['1100']
Z3exp = results['0011'] + results['0101'] - results['1001'] + results['0110'] - results['1010'] - results['1100']


1/2*(Z0exp - Z1exp)

-0.6666666666666661